In [60]:
#Requests Module for working with RESTful APIs
import requests
from requests_oauthlib import OAuth2Session
import importlib

#Formatting Packages for working with JSON
import json
import pprint
pp = pprint.PrettyPrinter(indent=2)

#Pandas Packages
import pandas as pd 
import geopandas as gpd

#writing shape files
import shapefile

In [76]:
def query_maker(years,key,variables):
    urls = [] 
    for year in years:
        url = "https://api.census.gov/data/" + \
                year + \
                "/acs/acs5?get="
        for variable in variables: 
            url = url + variable + "," 
        url = url +"NAME&for=tract:*&in=state:39%20county:049&key=" + api_key
        urls.append(url)
    responses = [requests.get(url) for url in urls] 
    queries = [json.loads(response.text) for response in responses] 
    return queries

In [83]:
def build_frame(query):
    df = pd.DataFrame(query)
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header
    df['NAMELSAD'], df['County_State'] = df['NAME'].str.split(',', 1).str
    df = df.drop(['NAME'],axis=1)
    df['County'], df['State'] = df['County_State'].str.split(',', 1).str
    df = df.drop(['County_State'],axis=1)
    return df

In [77]:
###Obtaining URLS for Census API 
years = ['2012','2017']
api_key = "6518d805990d22f00cb5e9573e86376f70425c04"
variables = ['B00001_001E','B25056_001E'] 
queries = query_maker(years,api_key,variables)

In [94]:
###2012
census_2012 = build_frame(queries[0])
census_2012['year'] = '2012'

###2017
census_2017 = build_frame(queries[1])
census_2017['year'] = '2017'

,B00001_001E,B25056_001E,state,county,tract,NAMELSAD,County,State,year
1,357,594,39,049,007811,Census Tract 78.11,Franklin County,Ohio,2017
2,272,997,39,049,002600,Census Tract 26,Franklin County,Ohio,2017
3,128,506,39,049,002800,Census Tract 28,Franklin County,Ohio,2017
4,241,1181,39,049,004900,Census Tract 49,Franklin County,Ohio,2017
5,298,964,39,049,005500,Census Tract 55,Franklin County,Ohio,2017


In [89]:
ohio = gpd.read_file('state_shape_files/tl_2018_39_tract.shp')
ohio = ohio[ohio['COUNTYFP']== '049']
ohio.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
13,39,049,006392,39049006392,63.92,Census Tract 63.92,G5020,S,4125349,61835,+40.1023122,-083.0452464,"POLYGON ((-83.057084 40.111397, -83.056659 40...."
14,39,049,006500,39049006500,65,Census Tract 65,G5020,S,2423382,28836,+40.0045889,-083.0676698,"POLYGON ((-83.083044 40.002847, -83.0828289999..."
15,39,049,006600,39049006600,66,Census Tract 66,G5020,S,2128916,26973,+39.9958355,-083.0609835,"POLYGON ((-83.07917399999999 39.997529, -83.07..."
16,39,049,006710,39049006710,67.10,Census Tract 67.10,G5020,S,1725268,2336,+40.0876281,-083.0127650,"POLYGON ((-83.022555 40.082836, -83.0225569999..."
17,39,049,006721,39049006721,67.21,Census Tract 67.21,G5020,S,2825390,36972,+40.1027983,-083.0233041,"POLYGON ((-83.035578 40.102302, -83.0355029999..."


In [95]:
###2012 
combined_2012 = pd.concat([census_2012.set_index('NAMELSAD'),ohio.set_index('NAMELSAD')], axis=1, join='inner')
combined_2012.reset_index(level=0, inplace=True)
combined_2012.head()
###2017 
combined_2017 = pd.concat([census_2017.set_index('NAMELSAD'),ohio.set_index('NAMELSAD')], axis=1, join='inner')
combined_2017.reset_index(level=0, inplace=True)
combined_2017.head()

,NAMELSAD,B00001_001E,B25056_001E,state,county,tract,County,State,year,STATEFP,...,TRACTCE,GEOID,NAME,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,Census Tract 78.11,357,594,39,049,007811,Franklin County,Ohio,2017,39,...,007811,39049007811,78.11,G5020,S,2193824,2283,+40.0347566,-083.0435104,"POLYGON ((-83.05030000000001 40.041497, -83.05..."
1,Census Tract 26,272,997,39,049,002600,Franklin County,Ohio,2017,39,...,002600,39049002600,26,G5020,S,3566865,21400,+39.9881814,-082.9252016,"POLYGON ((-82.94038399999999 39.981438, -82.94..."
2,Census Tract 28,128,506,39,049,002800,Franklin County,Ohio,2017,39,...,002800,39049002800,28,G5020,S,1142270,0,+39.9730499,-082.9648451,"POLYGON ((-82.97030199999999 39.974462, -82.96..."
3,Census Tract 49,241,1181,39,049,004900,Franklin County,Ohio,2017,39,...,004900,39049004900,49,G5020,S,2431848,0,+39.9483027,-083.0478959,"POLYGON ((-83.056764 39.955763, -83.0552709999..."
4,Census Tract 55,298,964,39,049,005500,Franklin County,Ohio,2017,39,...,005500,39049005500,55,G5020,S,1480787,0,+39.9460765,-082.9603156,"POLYGON ((-82.973241 39.944115, -82.9732169999..."


In [98]:
combined = [combined_2012,combined_2017]
combined = pd.concat(combined)
combined.head()

,NAMELSAD,B00001_001E,B25056_001E,state,county,tract,County,State,year,STATEFP,...,TRACTCE,GEOID,NAME,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,Census Tract 37,241,988,39,049,003700,Franklin County,Ohio,2012,39,...,003700,39049003700,37,G5020,S,2091019,40589,+39.9627013,-082.9535872,"POLYGON ((-82.96656999999999 39.958913, -82.96..."
1,Census Tract 38,138,702,39,049,003800,Franklin County,Ohio,2012,39,...,003800,39049003800,38,G5020,S,722586,0,+39.9628717,-082.9736000,"POLYGON ((-82.98183499999999 39.960688, -82.98..."
2,Census Tract 40,141,1691,39,049,004000,Franklin County,Ohio,2012,39,...,004000,39049004000,40,G5020,S,2284859,74359,+39.9578076,-082.9938952,"POLYGON ((-83.01181299999999 39.952759, -83.01..."
3,Census Tract 42,75,389,39,049,004200,Franklin County,Ohio,2012,39,...,004200,39049004200,42,G5020,S,1195971,173303,+39.9597484,-083.0124370,"POLYGON ((-83.019744 39.963137, -83.01961 39.9..."
4,Census Tract 43,309,2304,39,049,004300,Franklin County,Ohio,2012,39,...,004300,39049004300,43,G5020,S,5112403,1293311,+39.9710998,-083.0502723,"POLYGON ((-83.085824 39.994919, -83.0857599999..."


In [99]:
####2012
#combined_2012 = gpd.GeoDataFrame(combined_2012, geometry='geometry')
#combined_2012.to_file('MyGeometries_2012.shp', driver='ESRI Shapefile')

####2017 
#combined_2017 = gpd.GeoDataFrame(combined_2017, geometry='geometry')
#combined_2017.to_file('MyGeometries_2017.shp', driver='ESRI Shapefile')
combined = gpd.GeoDataFrame(combined, geometry='geometry')
combined.to_file('MyGeometries.shp', driver='ESRI Shapefile')